In [136]:
import pandas as pd
from scipy import stats
# Define the file path
file_path = r"G:\Shared drives\San Diego Taxpayers Educational Foundation\Data Team\Campaign Finance & Political Activity\Wage Datasets\K12 wage data\fullteach.csv"
# Load the dataset
k12 = pd.read_csv(file_path)

# Display the first few rows of the dataset to confirm it's loaded
k12.head()


,name,position,pay_and_benefits,year,dist,enrollment,pay_and_benefits_adjusted
0,A RICE-HEALY,SPEECH /LANGUAGE PATHOLOGIST,99517.0,2014,SAN DIEGO UNIFIED,130303,96514.708781
1,A RICE-HEALY,SPEECH /LANGUAGE PATHOLOGIST,114043.0,2016,SAN DIEGO UNIFIED,129380,109095.103651
2,A RICE-HEALY,SPEECH /LANGUAGE PATHOLOGIST,117526.0,2017,SAN DIEGO UNIFIED,128040,110081.855597
3,A RICE-HEALY,SPEECH /LANGUAGE PATHOLOGIST,115337.0,2018,SAN DIEGO UNIFIED,126400,105455.774542
4,A RICE-HEALY,SPEECH /LANGUAGE PATHOLOGIST,115337.0,2019,SAN DIEGO UNIFIED,124105,103578.948271


In [138]:
# Filter rows where 'position' contains 'Teacher' and 'Status' is 'FT'
teacher_ft_jobs = k12[(k12['position'].str.contains('Teacher', case=False, na=False))]

# Get unique values of 'position' from the filtered rows
unique_teacher_ft_jobs = teacher_ft_jobs['position'].unique()

# Print the list of unique job titles
print(unique_teacher_ft_jobs)


['TEACHER-DEAF ED' 'TEACHER' 'REGULAR TEACHER' ... 'RESOURCE TEACHER,RDM'
 'TEACHER BLK MT ACADEMY' 'OTAY TEACHER']


In [144]:
#Filter for FT jobs and teachers from unique_teacher_ft_jobs
fil_k12 = k12[(k12['position'].isin(unique_teacher_ft_jobs))]

In [149]:
# Reset the index to avoid issues with duplicate labels
fil_k12 = fil_k12.reset_index(drop=True)

# Create the 'switched' column that reflects whether the teacher switched districts even once
fil_k12['switched'] = fil_k12.groupby('name')['dist'].transform(lambda x: x.nunique() > 1)

# Show the first few rows of the updated dataset
fil_k12.head()


,name,position,pay_and_benefits,year,dist,enrollment,pay_and_benefits_adjusted,switched
0,"AABERG, SUSAN",TEACHER-DEAF ED,94992.00,2013,SAN DIEGO UNIFIED,130270,93620.682134,False
1,AAMNA AHMAD KAMIL,TEACHER,67045.93,2012,GROSSMONT UNION HIGH,23675,67045.930000,False
2,AAMNA AHMAD KAMIL,TEACHER,70615.32,2013,GROSSMONT UNION HIGH,22965,69595.907314,False
3,AAMNA AHMAD KAMIL,TEACHER,77772.07,2014,GROSSMONT UNION HIGH,22555,75425.793456,False
4,AAMNA AHMAD KAMIL,TEACHER,78886.21,2015,GROSSMONT UNION HIGH,22220,76415.617862,False


In [151]:
# Group by 'name' and calculate the difference between the most recent and first pay_and_benefits
fil_k12['pay_difference'] = fil_k12.groupby('name')['pay_and_benefits'].transform(lambda x: x.iloc[-1] - x.iloc[0])

# Filter out rows where there is only one entry of a name (i.e., those with no difference in years)
diff_k12 = fil_k12.groupby('name').filter(lambda x: len(x) > 1)

# Show the first few rows of the updated dataset
diff_k12.head()

,name,position,pay_and_benefits,year,dist,enrollment,pay_and_benefits_adjusted,switched,pay_difference
1,AAMNA AHMAD KAMIL,TEACHER,67045.93,2012,GROSSMONT UNION HIGH,23675,67045.930000,False,11840.28
2,AAMNA AHMAD KAMIL,TEACHER,70615.32,2013,GROSSMONT UNION HIGH,22965,69595.907314,False,11840.28
3,AAMNA AHMAD KAMIL,TEACHER,77772.07,2014,GROSSMONT UNION HIGH,22555,75425.793456,False,11840.28
4,AAMNA AHMAD KAMIL,TEACHER,78886.21,2015,GROSSMONT UNION HIGH,22220,76415.617862,False,11840.28
5,AAMNA KAMIL,TEACHER,87898.00,2016,GROSSMONT UNION HIGH,21860,84084.436754,False,46241.00


In [161]:
# Separate the data into two groups based on the 'switched' column
true_group = diff_k12[diff_k12['switched'] == True]['pay_difference']
false_group = diff_k12[diff_k12['switched'] == False]['pay_difference']

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(true_group, false_group)

print(f"T-statistic: {t_stat}")
print(f"P-value: {p_value}")


T-statistic: 38.330530804686305
P-value: 0.0
